Lab Report: Database Management in Python
Topic: SQLite,SQLAlchemy, and NoSQL Integration
Objectives
To implement a local relational database using SQLite.
To abstract database interactions using the SQLAlchemy ORM.
To understand document-oriented data storage using NoSQL (MongoDB).
Theory
Relational Databases (SQLite & SQLAlchemy)
SQLite is a C-language library that implements a small, fast, self-contained SQL database engine. It is "serverless," meaning the database is just a file on your disk.

SQLAlchemy is an Object-Relational Mapper (ORM). Instead of writing raw SQL strings like SELECT * FROM users, you interact with Python objects. This prevents SQL injection and makes the code more readable.

NoSQL (MongoDB) Unlike SQL databases that use tables and rows, NoSQL databases like MongoDB use collections and documents (similar to JSON/Python dictionaries). This allows for a "schemaless" design where different records can have different fields.

SQLite implementation:

In [4]:
import sqlite3
#connect(creates file if it doesn't exit)
conn=sqlite3.connect('example.db')
cursor=conn.cursor()
#create table with all sqlite data types
cursor.execute("""
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    age INTEGER,
    salary REAL,
    is_active BOOLEAN,
    bio TEXT,
    join_date DATETIME,
    profile_picture BLOB,
    misc_data NUMERIC,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
""")

In [5]:
# Insert example data into users table
cursor.execute("""
INSERT INTO users (name, age, salary, is_active, bio, join_date, profile_picture, misc_data)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
""", ("Alice", 30, 5500.50, 1, "Software engineer", "2026-01-05 05:00:00", None, 123.45))

cursor.execute("""
INSERT INTO users (name, age, salary, is_active, bio, join_date, profile_picture, misc_data)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
""", ("Bob", 28, 48000.00, 1, "Data analyst", "2026-01-04 06:30:00", None, 678.90))


In [6]:
# List of users to insert
users_to_insert = [
    ("Alice", 30, 55000.58, 1, "Software engineer", "2026-01-05 08:00:00", None, 123.45),
    ("Bob", 28, 48000.00, 1, "Data analyst", "2026-01-04 09:30:00", None, 678.99),
    ("Charlie", 35, 62000.75, 1, "Project manager", "2026-01-03 14:15:00", None, 234.56)
]

# Insert multiple records using executemany
cursor.executemany("""
INSERT INTO users (name, age, salary, is_active, bio, join_date, profile_picture, misc_data)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
""", users_to_insert)


In [7]:
# 4. READ: Fetch all records
print("All users:")

cursor.execute("SELECT * FROM users")
rows = cursor.fetchall()

for row in rows:
    print(row)


All users:
(1, 'Alice', 30, 5500.5, 1, 'Software engineer', '2026-01-05 05:00:00', None, 123.45, '2026-01-28 11:35:33')
(2, 'Bob', 28, 48000.0, 1, 'Data analyst', '2026-01-04 06:30:00', None, 678.9, '2026-01-28 11:35:33')
(3, 'Alice', 30, 55000.58, 1, 'Software engineer', '2026-01-05 08:00:00', None, 123.45, '2026-01-28 11:36:38')
(4, 'Bob', 28, 48000.0, 1, 'Data analyst', '2026-01-04 09:30:00', None, 678.99, '2026-01-28 11:36:38')
(5, 'Charlie', 35, 62000.75, 1, 'Project manager', '2026-01-03 14:15:00', None, 234.56, '2026-01-28 11:36:38')


In [8]:
# 5. UPDATE: Update salary for a user

# Update Alice's salary
cursor.execute("""
UPDATE users
SET salary = ?
WHERE name = ?
""", (70000.08, "Alice"))

print("\nAfter updating Alice's salary:")

# Fetch and display Alice's updated record
cursor.execute("SELECT * FROM users WHERE name = ?", ("Alice",))
print(cursor.fetchone())



After updating Alice's salary:
(1, 'Alice', 30, 70000.08, 1, 'Software engineer', '2026-01-05 05:00:00', None, 123.45, '2026-01-28 11:35:33')


In [9]:
# 6. DELETE: Delete a user

# Delete Bob from the users table
cursor.execute("DELETE FROM users WHERE name = ?", ("Bob",))

print("\nAfter deleting Bob:")

# Fetch and display all remaining users
cursor.execute("SELECT * FROM users")
rows = cursor.fetchall()
for row in rows:
    print(row)

# 7. Commit changes and close
conn.commit()
conn.close()



After deleting Bob:
(1, 'Alice', 30, 70000.08, 1, 'Software engineer', '2026-01-05 05:00:00', None, 123.45, '2026-01-28 11:35:33')
(3, 'Alice', 30, 70000.08, 1, 'Software engineer', '2026-01-05 08:00:00', None, 123.45, '2026-01-28 11:36:38')
(5, 'Charlie', 35, 62000.75, 1, 'Project manager', '2026-01-03 14:15:00', None, 234.56, '2026-01-28 11:36:38')


SQLAlchemy

In [ ]:
#  Import required modules
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import declarative_base, sessionmaker

#  Setup the connection (engine)
engine = create_engine("sqlite:///datal.db", echo=True)

#. Create a base class
Base = declarative_base()

# Define the User table (model)
class User(Base):
    __tablename__ = "users"

    id = Column(Integer, primary_key=True)
    name = Column(String)
    email = Column(String)
    course = Column(String)

# . Create the table in the database
Base.metadata.create_all(engine)


2026-01-28 17:34:47,426 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-01-28 17:34:47,427 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2026-01-28 17:34:47,428 INFO sqlalchemy.engine.Engine [raw sql] ()
2026-01-28 17:34:47,431 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2026-01-28 17:34:47,436 INFO sqlalchemy.engine.Engine [raw sql] ()
2026-01-28 17:34:47,439 INFO sqlalchemy.engine.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	email VARCHAR, 
	course VARCHAR, 
	PRIMARY KEY (id)
)


2026-01-28 17:34:47,441 INFO sqlalchemy.engine.Engine [no key 0.00192s] ()
2026-01-28 17:34:47,454 INFO sqlalchemy.engine.Engine COMMIT


In [12]:
#  Create session
Session = sessionmaker(bind=engine)
session = Session()

# Multiple rows to insert
users = [
    User(name="BE Computer", email="befncit.edu", course="Engineering"),
    User(name="BSc CSIT", email="csit@ncit.edu", course="Science"),
    User(name="B11", email="bit@ncit.edu", course="IT")
]

# Add all users to the session
session.add_all(users)

# Commit the changes to the database
session.commit()


2026-01-28 17:36:56,581 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-01-28 17:36:56,587 INFO sqlalchemy.engine.Engine INSERT INTO users (name, email, course) VALUES (?, ?, ?) RETURNING id
2026-01-28 17:36:56,589 INFO sqlalchemy.engine.Engine [generated in 0.00034s (insertmanyvalues) 1/3 (ordered; batch not supported)] ('BE Computer', 'befncit.edu', 'Engineering')
2026-01-28 17:36:56,594 INFO sqlalchemy.engine.Engine INSERT INTO users (name, email, course) VALUES (?, ?, ?) RETURNING id
2026-01-28 17:36:56,596 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/3 (ordered; batch not supported)] ('BSc CSIT', 'csit@ncit.edu', 'Science')
2026-01-28 17:36:56,599 INFO sqlalchemy.engine.Engine INSERT INTO users (name, email, course) VALUES (?, ?, ?) RETURNING id
2026-01-28 17:36:56,600 INFO sqlalchemy.engine.Engine [insertmanyvalues 3/3 (ordered; batch not supported)] ('B11', 'bit@ncit.edu', 'IT')
2026-01-28 17:36:56,605 INFO sqlalchemy.engine.Engine COMMIT


In [13]:
# Insert a single user
user = User(
    name="BE Software",
    email="software@ncit.edu",
    course="Engineering"
)

session.add(user)
session.commit()


2026-01-28 17:39:21,717 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-01-28 17:39:21,721 INFO sqlalchemy.engine.Engine INSERT INTO users (name, email, course) VALUES (?, ?, ?)
2026-01-28 17:39:21,724 INFO sqlalchemy.engine.Engine [generated in 0.00395s] ('BE Software', 'software@ncit.edu', 'Engineering')
2026-01-28 17:39:21,728 INFO sqlalchemy.engine.Engine COMMIT


In [14]:
# Fetch all users from the table
all_users = session.query(User).all()

# Loop through each user and print details
for user in all_users:
    print(user.id, user.name, user.email, user.course)


2026-01-28 17:41:13,059 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-01-28 17:41:13,065 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.email AS users_email, users.course AS users_course 
FROM users
2026-01-28 17:41:13,068 INFO sqlalchemy.engine.Engine [generated in 0.00249s] ()
1 BE Computer befncit.edu Engineering
2 BSc CSIT csit@ncit.edu Science
3 B11 bit@ncit.edu IT
4 BE Software software@ncit.edu Engineering


In [15]:
# Fetch a single user by name
user = session.query(User).filter_by(name="B11").first()  # Make sure the name matches exactly

if user:
    print(user.name, user.email)
else:
    print("User not found")


2026-01-28 17:42:08,364 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.email AS users_email, users.course AS users_course 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2026-01-28 17:42:08,366 INFO sqlalchemy.engine.Engine [generated in 0.00228s] ('B11', 1, 0)
B11 bit@ncit.edu


In [ ]:
# Fetch a single user by name
user = session.query(User).filter_by(name="B11").first()  

# Update the user's course if they exist
if user:
    user.course = "Information Technology"
    session.commit()
    print(f"Updated {user.name}'s course to {user.course}")
else:
    print("User not found")


2026-01-28 17:47:41,183 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.email AS users_email, users.course AS users_course 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2026-01-28 17:47:41,186 INFO sqlalchemy.engine.Engine [cached since 332.8s ago] ('B11', 1, 0)
2026-01-28 17:47:41,193 INFO sqlalchemy.engine.Engine UPDATE users SET course=? WHERE users.id = ?
2026-01-28 17:47:41,195 INFO sqlalchemy.engine.Engine [generated in 0.00173s] ('Information Technology', 3)
2026-01-28 17:47:41,199 INFO sqlalchemy.engine.Engine COMMIT
2026-01-28 17:47:41,235 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-01-28 17:47:41,239 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.email AS users_email, users.course AS users_course 
FROM users 
WHERE users.id = ?
2026-01-28 17:47:41,242 INFO sqlalchemy.engine.Engine [generated in 0.00358s] (3,)
Updated B11's course to Information Technology


In [17]:
# Update all users whose course is "Engineering"
session.query(User).filter(User.course == "Engineering").update(
    {"course": "BSc Engineering"}, synchronize_session=False
)

# Commit the changes
session.commit()


2026-01-28 17:48:36,972 INFO sqlalchemy.engine.Engine UPDATE users SET course=? WHERE users.course = ?
2026-01-28 17:48:36,976 INFO sqlalchemy.engine.Engine [generated in 0.00417s] ('BSc Engineering', 'Engineering')
2026-01-28 17:48:36,988 INFO sqlalchemy.engine.Engine COMMIT


In [18]:
# Fetch the user to delete
user = session.query(User).filter_by(name="BSc CSIT").first()

# Delete the user if they exist
if user:
    session.delete(user)
    session.commit()
    print(f"Deleted user {user.name}")
else:
    print("User not found")


2026-01-28 17:49:17,394 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-01-28 17:49:17,397 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.email AS users_email, users.course AS users_course 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2026-01-28 17:49:17,399 INFO sqlalchemy.engine.Engine [cached since 429s ago] ('BSc CSIT', 1, 0)
2026-01-28 17:49:17,404 INFO sqlalchemy.engine.Engine DELETE FROM users WHERE users.id = ?
2026-01-28 17:49:17,406 INFO sqlalchemy.engine.Engine [generated in 0.00205s] (2,)
2026-01-28 17:49:17,411 INFO sqlalchemy.engine.Engine COMMIT
Deleted user BSc CSIT


In [19]:
# Delete all users whose course is "IT"
session.query(User).filter(User.course == "IT").delete(synchronize_session=False)

# Commit the changes
session.commit()


2026-01-28 17:53:19,351 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-01-28 17:53:19,355 INFO sqlalchemy.engine.Engine DELETE FROM users WHERE users.course = ?
2026-01-28 17:53:19,357 INFO sqlalchemy.engine.Engine [generated in 0.00241s] ('IT',)
2026-01-28 17:53:19,361 INFO sqlalchemy.engine.Engine COMMIT


NoSQL with MongoDB

In [ ]:
from pymongo import MongoClient

#  Connect to MongoDB server (local)
client = MongoClient("mongodb://localhost:27017/") 

# Access database and collection
db = client["my_database"]
users = db["users_collection"]


In [ ]:
#  Insert a single document
users.insert_one({
    "id": 1,
    "name": "Alice",
    "age": 30,
    "salary": 53008.39,
    "is_active": True,
    "bio": "Software engineer",
    "skills": ["Python", "AI"],
    "join_date": "2026-01-05"
})

# Insert multiple documents
users_to_insert = [
    {
        "id": 2,
        "name": "Bob",
        "age": 25,
        "salary": 48060.09,
        "is_active": False,
        "bio": "Data analyst",
        "skills": ["SQL", "Excel"],
        "join_date": "2026-01-04"
    },
    {
        "id": 3,
        "name": "Charlie",
        "age": 35,
        "salary": 52630.75,
        "is_active": True,
        "bio": "Project manager",
        "skills": ["Management", "Leadership"],
        "join_date": "2026-01-03"
    }
]

users.insert_many(users_to_insert)


In [ ]:
#  Find all users
print("All users:")
for user in users.find():
    print(user)

# Find one user
alice = users.find_one({"name": "Alice"})
print("\nFind Alice:")
print(alice)


In [ ]:
# Update Alice's salary
users.update_one(
    {"name": "Alice"},         # Filter: find the user by name
    {"$set": {"salary": 70600.11}}  # Update: set salary to new value
)

# Print updated document
print("\nAfter updating Alice's salary:")
alice = users.find_one({"name": "Alice"})
print(alice)


In [ ]:
# Delete Bob
users.delete_one({"name": "Bob"})

print("\nAfter deleting Bob:")

# Print all remaining users
for user in users.find():
    print(user)
